In [177]:
from transformers import (
    AutoConfig, BertConfig, BertTokenizer, BertForRetrieval,
    BertForSequenceClassification)
import torch
import numpy as np
import pickle

In [178]:
import pandas as pd

In [179]:
# csvf = StringIO(csv_file.read().decode('utf-8-sig'))

In [180]:
df = pd.read_csv('/home/transformers-public/data_dir/doordash_test.csv')

In [181]:
df

,Title,Description,Manual Cuisine,Manual Dish Type,cuisine_label,dish_label
0,BBQ Pork Bao,Char-siu roasted pork belly,Chinese,Other,chinese,other
1,Seafood Gyoza,"Spinach skin, crab, scallop, shrimp, chilli, b...",Japanese,Other,japanese,other
2,Impossible MeltBurger,Our signature meltburger made with a plant-bas...,American,Burger,american,burger
3,Steak & Cheese,"Grilled steak, provolone, melted cheddar, cara...",American,Sandwich,american,sandwich
4,Mission,"Romaine hearts, cabbage, green chili chicken, ...",Mexican,Salad,mexican,salad
...,...,...,...,...,...,...
67,Southwest BBQ Chicken,"Chargrilled chicken breast atop garden greens,...",other,salad,other,salad
68,Santa Barbara Cobb,"Crisp shredded iceberg and Romaine lettuce, di...",other,salad,other,salad
69,1191 Maguro Sashimi,tuna,japanese,sushi,japanese,sushi
70,Salsiccia,"House-made sicilian sausage, pickled jalapeños...",NaN,NaN,italian,pizza


In [182]:
titles = df.Title.values
cuisine_label = df.cuisine_label.values
dish_label = df.dish_label.values

# titles = ['tomato basil', 'clam chowder', 'chicken noodle']
# dish_label = ['soup' for _ in range(3)]

In [200]:
model_dir_pattern = "/home/transformers-public/output_dirs/{run_name}/checkpoint-{ckpt_num}"
# test_long_new_dataloader/checkpoint-3000"
model_dir = model_dir_pattern.format(run_name="test_new_cleaned_loader",
                                     ckpt_num=70000)

model_dir = model_dir_pattern.format(run_name="dish_titles_better",
                                     ckpt_num=8000)
config = AutoConfig.from_pretrained(model_dir)
tokenizer = BertTokenizer.from_pretrained(model_dir, do_lower_case=True)
model = BertForSequenceClassification.from_pretrained(model_dir, from_tf=False, config=config,)

In [201]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
device = "cpu"

cuda


In [202]:
model = model.to(device)

In [203]:
max_seq_length=64
test_input = tokenizer.batch_encode_plus(
    titles,
    max_length=max_seq_length,
    pad_to_max_length=True,
)

In [204]:
print(test_input)

{'input_ids': [[101, 170, 10457, 11703, 10183, 10174, 18323, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 14931, 21330, 12680, 175, 15594, 10637, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 38199, 10911, 11533, 59692, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 28780, 43213, 111, 10262, 13565, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 17032, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [205]:
model_inp = dict(test_input)
model_inp['labels'] = [0] * len(test_input['input_ids'])  # dummy labels
model_inp = {k: torch.tensor(v).to(device) for k, v in model_inp.items()}
model_out = model(**model_inp)

In [206]:
len(model_out)

3

In [207]:
print([x.shape for x in model_out])  # loss, logits, pooled_output (i.e. encoding)

[torch.Size([]), torch.Size([72, 8]), torch.Size([72, 768])]


In [208]:
import scipy.special

In [209]:
predicted_logits = model_out[1].detach().cpu().numpy()
predicted_class = np.argmax(predicted_logits, axis=1)
pred_scores = scipy.special.softmax(predicted_logits, axis=1)

In [210]:
print(predicted_logits)

[[-3.30321342e-01  3.33981371e+00 -3.24024498e-01 -2.83861697e-01
   2.29354572e+00  1.14306450e+00 -2.21053910e+00 -7.96379709e+00]
 [-1.15316749e+00  1.69616532e+00 -1.05231953e+00 -1.51150182e-01
  -9.26479578e-01  2.02527976e+00  2.64831042e+00 -5.16580153e+00]
 [ 4.60723639e+00  7.69482374e-01 -2.90989900e+00 -5.86271107e-01
   3.71357036e+00 -2.02019668e+00 -1.03562319e+00 -6.60482597e+00]
 [ 1.74086452e-01  1.21429014e+00  1.99506271e+00  9.68889296e-01
   3.46122503e+00 -1.35306692e+00 -3.08116007e+00 -8.02219582e+00]
 [ 1.31311297e+00  2.17544585e-01  1.27988839e+00  1.03779268e+00
   1.02066541e+00 -1.96922684e+00  3.55277717e-01 -6.42019510e+00]
 [-2.24870324e+00  6.04797268e+00 -2.41203919e-01  1.60760474e+00
  -7.13477612e-01  1.76029071e-01 -1.54521883e+00 -6.65775204e+00]
 [-2.55595922e+00  4.51193428e+00 -7.72419214e-01  3.95828819e+00
   1.72040892e+00 -5.82091153e-01 -2.46904159e+00 -8.30620956e+00]
 [-1.21765006e+00  2.09973860e+00 -1.42707813e+00  4.76869345e-02
  -

In [211]:
print(predicted_class)

[1 6 0 4 0 1 1 5 1 1 1 6 1 1 1 5 3 5 1 1 2 1 1 1 1 1 1 6 6 5 6 3 1 1 6 4 1
 1 4 1 1 5 1 1 1 0 1 5 1 0 4 5 3 0 4 4 4 4 1 2 4 5 4 3 5 3 2 3 3 6 2 2]


In [212]:
cuisines = ['american', 'chinese', 'indian', 'italian', 'japanese', 'mexican', 'other', 'thai']
print(len(cuisines))

cuisines = ['burger', 'other', 'pizza', 'salad', 'sandwich', 'soup', 'sushi']

8


In [213]:
# cuisines = pickle.load(open('../data_dir/cuisines_ordered_list.pkl', 'rb'))

In [214]:
wrong = 0
for i in range(len(titles)):
    pred_label = cuisines[predicted_class[i]]
    true_label = dish_label[i]  # cuisine_label[i]
    title = titles[i]
    if pred_label != true_label:
        wrong += 1
        print("WRONG", end=" : ")
    print(title, 'true:', true_label, 'pred:', pred_label,
         'score:', pred_scores[i][predicted_class[i]])
print(wrong/len(titles))

BBQ Pork Bao true: other pred: other score: 0.6476265
WRONG : Seafood Gyoza true: other pred: sushi score: 0.48578048
Impossible MeltBurger true: burger pred: burger score: 0.6936396
Steak & Cheese true: sandwich pred: sandwich score: 0.6820474
WRONG : Mission true: salad pred: burger score: 0.2364549
Pacifica Grain Bowl true: other pred: other score: 0.98195255
WRONG : Tea Leaf Salad-Laphet true: salad pred: other score: 0.6063673
WRONG : Spicy Noodles true: other pred: soup score: 0.9622392
Burmese Tomato Chutney true: other pred: other score: 0.5856747
Tikka Masala Burrito true: other pred: other score: 0.9830263
Naughty Naan true: other pred: other score: 0.9925104
WRONG : Samosas true: other pred: sushi score: 0.45502675
Super Meat Nachos true: other pred: other score: 0.85452145
Jumbo Wings (10-pc Regular Pack) true: other pred: other score: 0.95980406
Pad-King Rice Plate (Lunch) true: other pred: other score: 0.9613848
Tom Yum Goong (Spicy Shrimp Lemongrass Soup) true: soup pred

In [54]:
[('predicted', 'title', 'actual')] + [
    (cuisines[p], title, true)  for p, title, a in zip(predicted_class, titles, cuisine_label)]

NameError: name 'true' is not defined

## Argmax only over desired labels

In [107]:
cuisine_label_int = [cuisines.index(c) for c in cuisine_label]

In [108]:
target_label_set = list(sorted(set(cuisine_label_int)))
print(target_label_set)

[1, 7, 15, 17, 18, 22, 24, 28]


In [109]:
predicted_logits.shape

(72, 30)

In [112]:
pred_logits_restricted = predicted_logits[:, target_label_set]
pred_logits_restricted.shape
predicted_class_restricted = np.argmax(pred_logits_restricted, axis=1)

In [113]:
wrong = 0
for i in range(len(titles)):
    pcr = predicted_class_restricted[i]
    pred_label = cuisines[target_label_set[pcr]]
    true_label = cuisine_label[i]
    title = titles[i]
    if pred_label != true_label:
        wrong += 1
        print(title, 'true:', true_label, 'pred:', pred_label)
print(wrong/len(titles))

BBQ Pork Bao true: chinese pred: japanese
Steak & Cheese true: american pred: italian
Mission true: mexican pred: japanese
Spicy Noodles true: thai pred: japanese
Tikka Masala Burrito true: indian pred: mexican
Samosas true: indian pred: japanese
Jumbo Wings (10-pc Regular Pack) true: american pred: italian
Tom Yum Goong (Spicy Shrimp Lemongrass Soup) true: thai pred: japanese
Pad Woon Sen true: thai pred: japanese
Malabar Parota true: indian pred: other
The Meats true: italian pred: other
Vegetable Briyani true: indian pred: japanese
Turkey Breast Footlong Regular Sub true: other pred: american
Buffalo Chicken Sandwich true: american pred: other
Tingly Mapo Tofu true: chinese pred: japanese
Wonton Soup true: chinese pred: japanese
Nashville Looks So Good on You true: american pred: other
Kimchee Stew true: other pred: japanese
Vegan Caesar true: other pred: mexican
McCovey true: american pred: other
Gus's Special true: american pred: mexican
Daily Lentil Soup true: other pred: america

In [116]:
wrong

26

In [117]:
len(titles)

72